# Predicting Age From X-Rays

# Goal
Develop an algorithm to determine the age of a child by utilizing x-rays of hands (pediatric hand radiographs).

In [8]:
# Common variables for all models
from keras.callbacks import EarlyStopping, ReduceLROnPlateau    

epochs = 250

batchSize = 20
imageLength = 224
imageHeight = 224

# Stop training when a monitored quantity has stopped improving after 25 epochs
earlyStop = EarlyStopping(patience=25, verbose=1)

# Reduce learning rate when a metric has stopped improving
reduceLR = ReduceLROnPlateau(factor=0.3, patience=3, cooldown=3, verbose=1)

## Expore the dataset

In [9]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline 

# CSV file contains the id, age, and gender related to each x-ray
dataset = pd.read_csv("Input/boneage-dataset.csv")

# Dropping all rows with any NA values
dataset.dropna()

print('There are %d total data samples.' % dataset.shape[0])
print('\nData type for each column is:\n',dataset.dtypes)
dataset.head()

There are 12611 total data samples.
('\nData type for each column is:\n', id         int64
boneage    int64
male        bool
dtype: object)


,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False


### Connect the x-ray images to the dataset

In [10]:
import os

dataset['image_path'] = dataset['id'].map(lambda x: os.path.join('Input/boneage-dataset', '{}.png'.format(x)))

if dataset['image_path'].map(os.path.exists).sum() != dataset.shape[0]:
    print('Warning!! Only', dataset['image_path'].map(os.path.exists).sum(), 
          'x-ray images found out of', dataset.shape[0], 'total! \nWill continue with reduced dataset!!')
    dataset = dataset[dataset['image_path'].map(os.path.exists)]
else:
    print('Awesome!! All',dataset['image_path'].map(os.path.exists).sum(), 
          'x-ray images found out of', dataset.shape[0], 'total!')

('Awesome!! All', 12611, 'x-ray images found out of', 12611, 'total!')


### Split data in a stratified fashion

In [11]:
# Make new colum 'boneage_category' such that each category has the same number of samples
deciles_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
deciles = dataset['boneage'].quantile(q=deciles_list)

deciles_labels =[]
for i in range(deciles.size):
    if i==deciles.size-1: continue
    deciles_labels.append(str(deciles[deciles.index[i]])+'-'+str(deciles[deciles.index[i+1]]))   
print(deciles_labels, '\n')
    
dataset['boneage_category'] = pd.qcut(dataset['boneage'], q=deciles_list, labels=deciles_labels)
print(dataset.head())

(['1.0-69.0', '69.0-94.0', '94.0-106.0', '106.0-120.0', '120.0-132.0', '132.0-144.0', '144.0-156.0', '156.0-162.0', '162.0-168.0', '168.0-228.0'], '\n')
     id  boneage   male                      image_path boneage_category
0  1377      180  False  Input/boneage-dataset/1377.png      168.0-228.0
1  1378       12  False  Input/boneage-dataset/1378.png         1.0-69.0
2  1379       94  False  Input/boneage-dataset/1379.png        69.0-94.0
3  1380      120   True  Input/boneage-dataset/1380.png      106.0-120.0
4  1381       82  False  Input/boneage-dataset/1381.png        69.0-94.0


### Split data into training, testing, and validation sets

In [12]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dataset, test_size=0.25, random_state=99, 
                                                    stratify=dataset['boneage_category'])
dataset_train, dataset_val  = train_test_split(dataset_train, test_size=0.25, random_state=99,
                                                    stratify=dataset_train['boneage_category'])
                                                                    
print("The dataset has a training set of %d examples." % len(dataset_train))
print("The dataset has a test set of %d examples." % len(dataset_test))
print("The dataset has a validation set of %d examples." % len(dataset_val))

The dataset has a training set of 7093 examples.
The dataset has a test set of 3153 examples.
The dataset has a validation set of 2365 examples.


### Create and configure augmented image generator

In [14]:
from keras.preprocessing.image import ImageDataGenerator

# Generate batches of tensor image data with real-time data augmentation
datagen_train = ImageDataGenerator(
    width_shift_range=0.1,    # Fraction of total width for random horizontal shifts
    height_shift_range=0.1,   # Fraction of total height for random vertical shifts
    rotation_range = 5,        # Degree range for random rotations
    horizontal_flip=True)      # Randomly flip images horizontally
datagen_test = ImageDataGenerator(rescale=1./255)
datagen_val = ImageDataGenerator(rescale=1./255)

generator_train = datagen_train.flow_from_directory(
        os.path.dirname(dataset_train['image_path'].values[0]),  # this is the target directory
        target_size=(imageLength, imageHeight),
        batch_size=batchSize,
        class_mode='sparse')
generator_train.filenames = dataset_train['image_path'].values
generator_train.classes = np.stack(dataset_train['boneage'].values)
generator_train.samples = dataset_train.shape[0]
generator_train.n = dataset_train.shape[0]
generator_train._set_index_array()
generator_train.directory = ''

generator_val = datagen_val.flow_from_directory(
        os.path.dirname(dataset_val['image_path'].values[0]),  # this is the target directory
        target_size=(imageLength, imageHeight),
        batch_size=batchSize,
        class_mode='sparse')
generator_val.filenames = dataset_val['image_path'].values
generator_val.classes = np.stack(dataset_val['boneage'].values)
generator_val.samples = dataset_val.shape[0]
generator_val.n = dataset_val.shape[0]
#generator_val._set_index_array()
generator_val.directory = ''

generator_test = datagen_test.flow_from_directory(
        os.path.dirname(dataset_test['image_path'].values[0]),  # this is the target directory
        target_size=(imageLength, imageHeight),
        batch_size=batchSize,
        class_mode='sparse')
generator_test.filenames = dataset_test['image_path'].values
generator_test.classes = np.stack(dataset_test['boneage'].values)
generator_test.samples = dataset_test.shape[0]
generator_test.n = dataset_test.shape[0]
#generator_test._set_index_array()
generator_test.directory = ''

X_train, y_train = next(generator_train)
#X_test, y_test = next(generator_test)

Found 0 images belonging to 0 classes.


AttributeError: 'DirectoryIterator' object has no attribute '_set_index_array'

### Visualize original and augmented images

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

benchmarkModel = Sequential()

benchmarkModel.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu',
                          input_shape=(imageLength, imageHeight, 3)))
benchmarkModel.add(MaxPooling2D(pool_size=2))
benchmarkModel.add(Dropout(0.5))

benchmarkModel.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
benchmarkModel.add(MaxPooling2D(pool_size=2))
benchmarkModel.add(Dropout(0.5))

benchmarkModel.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
benchmarkModel.add(MaxPooling2D(pool_size=2))
benchmarkModel.add(Dropout(0.5))

benchmarkModel.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
benchmarkModel.add(MaxPooling2D(pool_size=2))
benchmarkModel.add(Dropout(0.5))

benchmarkModel.add(Flatten())         
benchmarkModel.add(Dense(500, activation='relu'))
benchmarkModel.add(Dropout(0.5))

benchmarkModel.add(Dense(229, activation='softmax'))

benchmarkModel.summary()

### Compile the code

In [ ]:
benchmarkModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Calculate the classification accuracy of the benchmark model (before training)

In [ ]:
# Evaluate the test accuracy
accuracy = benchmarkModel.predict_generator(generator_test, 10)
#accuracy = 100*score[1]

# Print the test accuracy
print('Test accuracy of the benchmark model (before training): %.4f%%' % accuracy)

### Train the benchmark model

In [ ]:
from keras.callbacks import ModelCheckpoint    

# Save the best model after every epoch
checkPoint = ModelCheckpoint(filepath='SavedModels/BenchmarkBest.hdf5', save_best_only=True, verbose=1)

# Fit the model on batches with real-time data augmentation:
history = benchmarkModel.fit_generator(generator_train,
                                       steps_per_epoch=dataset_train.shape[0] // batchSize, epochs=epochs, 
                                       validation_data=generator_val, validation_steps=dataset_val.shape[0] // batchSize,
                                       callbacks=[checkPoint, earlyStop, reduceLR], verbose=0)
# List all data in history
print(history.history.keys())

# Summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Benchmark model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.savefig('Plots/BenchmarkAccuracy.pdf')
plt.show()

# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Benchmark model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.savefig('Plots/BenchmarkLoss.pdf')
plt.show()

### Calculate the classification accuracy of the benchmark model (after training)

In [ ]:
# Load the model with the best classification accuracy on the validation set
benchmarkModel.load_weights('SavedModels/BenchmarkBest.hdf5')

# Calculate the classification accuracy on the test set
score = benchmarkModel.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

# Print the test accuracy
print('Test accuracy of the benchmark model (after training): %.4f%%' % accuracy)

### Visualize the result of the benchmark model 

In [ ]:
y_pred = benchmarkModel.predict(X_test, verbose = False)

y_p = []
for y in y_pred:
    y_p.append(np.argmax(y))
y_t = []
for t in y_test:
    y_t.append(np.argmax(t))

fig, ax1 = plt.subplots(1,1, figsize = (6,6))
ax1.plot(y_t, y_t, 'b-', label = 'True')
ax1.plot(y_t, y_p, 'r.', label = 'Predictions')
ax1.legend()
ax1.set_title('Benchmark model', fontsize=25)
ax1.set_xlabel('True Age (Months)', fontsize=20)
ax1.set_ylabel('Predicted Age (Months)', fontsize=20)
pt.setTicks(ax1)

fig.savefig('Plots/BenchmarkResult.pdf')